# Context Graph Demo - AWS SageMaker Setup

This notebook helps you set up and run the Context Graph Demo on AWS SageMaker Labs.

## Step 1: Install Dependencies

In [5]:
!pip install python-dotenv faker neo4j boto3

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached faker-40.5.1-py3-none-any.whl.metadata (16 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
Using cached faker-40.5.1-py3-none-any.whl (2.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [faker]32m1/2 [faker]


In [2]:
!pip install -r requirements.txt

## Step 2: Test Neo4j Connection

In [26]:
from neo4j import GraphDatabase

# Your Neo4j credentials
uri = "neo4j+s://1cc5d8d5.databases.neo4j.io"
username = "neo4j"
password = "a8DQ6lCKS9R2xLjgXs22vQOhcFphBxwkPW_ZppoGOPE"

# Test connection
driver = GraphDatabase.driver(uri, auth=(username, password))
driver.verify_connectivity()
print(" Connected to Neo4j successfully!")
driver.close()

 Connected to Neo4j successfully!


## Step 3: Generate Sample Data

This will create realistic financial data in your Neo4j database.

In [6]:
import sys
sys.path.append('backend')

from scripts.generate_sample_data import DataGenerator

# Create generator with direct credentials
generator = DataGenerator(
    uri="neo4j+s://1cc5d8d5.databases.neo4j.io",
    username="neo4j",
    password="a8DQ6lCKS9R2xLjgXs22vQOhcFphBxwkPW_ZppoGOPE",
    database="neo4j"
)

try:
    generator.generate_all()
    print(" Sample data generated successfully!")
finally:
    generator.close()


CONTEXT GRAPH SAMPLE DATA GENERATOR

Clearing existing data...
Database cleared.
Creating constraints and indexes...
Constraints and indexes created.
Generating 50 organizations...
Created 50 organizations.
Generating 30 employees...
Created 30 employees.
Generating 15 policies...
Created 15 policies.
Generating 200 persons...
Created 200 persons.
Generating 350 accounts...
Created 350 accounts.
Generating 2000 transactions...
Created 2000 transactions.
Generating 600 decisions...
Created 600 decisions.
Generating 80 alerts...
Created 80 alerts.
Generating 100 support tickets...
Created 100 support tickets.
Creating causal chains between decisions...
Created causal chains.
Connecting organizations to the graph...
Connected organizations to the graph.

DATA GENERATION COMPLETE!

Generated:
  - 50 organizations (connected to persons, accounts, transactions)
  - 30 employees
  - 15 policies
  - 200 persons
  - 350 accounts
  - 2000 transactions
  - 600 decisions (with causal chains)
  - 

## Step 4: Test Bedrock Connection

Verify that Bedrock is accessible from SageMaker.

In [19]:
import boto3
import json

# Test Bedrock Claude
bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')

response = bedrock.invoke_model(
    modelId='us.anthropic.claude-sonnet-4-5-20250929-v1:0',
    body=json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 100,
        "messages": [
            {"role": "user", "content": "Say hello!"}
        ]
    })
)

result = json.loads(response['body'].read())
print("Bedrock Claude response:")
print(result['content'][0]['text'])

Bedrock Claude response:
Hello! 👋 How can I help you today?


## Step 5: Test Bedrock Embeddings

In [23]:
# Test Titan Embeddings
response = bedrock.invoke_model(
    modelId='amazon.titan-embed-text-v2:0',
    body=json.dumps({
        "inputText": "This is a test embedding"
    })
)

result = json.loads(response['body'].read())
embedding = result['embedding']
print(f" Bedrock Embeddings working! Dimension: {len(embedding)}")

 Bedrock Embeddings working! Dimension: 1024


## Step 6: Create .env File for Backend

In [9]:
env_content = """# Neo4j Connection
NEO4J_URI=neo4j+s://1cc5d8d5.databases.neo4j.io
NEO4J_USERNAME=neo4j
NEO4J_PASSWORD=a8DQ6lCKS9R2xLjgXs22vQOhcFphBxwkPW_ZppoGOPE
NEO4J_DATABASE=neo4j

# Bedrock Configuration
USE_BEDROCK=true
AWS_REGION=us-east-1

# AWS credentials (leave blank to use SageMaker IAM role)
AWS_ACCESS_KEY_ID=
AWS_SECRET_ACCESS_KEY=
AWS_SESSION_TOKEN=

# Bedrock Models
BEDROCK_CLAUDE_MODEL_ID=anthropic.claude-3-5-sonnet-20241022-v2:0
BEDROCK_EMBEDDING_MODEL_ID=amazon.titan-embed-text-v2:0
BEDROCK_EMBEDDING_DIMENSIONS=1024
"""

with open('.env', 'w') as f:
    f.write(env_content)

print(" .env file created!")

 .env file created!


## Step 7: Start the Backend Server

Run this in a terminal:

```bash
cd backend
uvicorn app.main:app --host 0.0.0.0 --port 8000
```

Or run it in the background from this notebook:

In [24]:
import subprocess
import time

# Start server in background
process = subprocess.Popen(
    ['uvicorn', 'app.main:app', '--host', '0.0.0.0', '--port', '8000'],
    cwd='backend',
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

print("Starting server...")
time.sleep(5)
print(" Server should be running on port 8000")
print("\nTo stop the server later, run: process.terminate()")

Starting server...
 Server should be running on port 8000

To stop the server later, run: process.terminate()


## Step 8: Test the API

In [39]:
import requests

base_url = 'http://localhost:8003/api'

# 1. Test graph statistics
print("=== Graph Statistics ===")
stats = requests.get(f'{base_url}/graph/statistics')
print(stats.json())

# 2. List decisions
print("\n=== Recent Decisions ===")
decisions = requests.get(f'{base_url}/decisions?limit=5')
print(decisions.json())

# 3. Search customers directly (bypassing agent)
print("\n=== Search Customers ===")
customers = requests.get(f'{base_url}/customers/search?query=&limit=5')
print(customers.json())

print("\n=== Graph Data ===")
graph = requests.get(f'{base_url}/graph?limit=20')
graph_data = graph.json()
print(f"Nodes: {len(graph_data['nodes'])}, Relationships: {len(graph_data['relationships'])}")

# 5. Semantic search for decisions
print("\n=== Semantic Search (Bedrock Embeddings) ===")
search = requests.get(f'{base_url}/search/decisions?query=high risk fraud&limit=3')
print(search.json())


=== Graph Statistics ===
{'node_counts': {'Transaction': 2000, 'Decision': 600, 'Alert': 80, 'SupportTicket': 100, 'Organization': 50, 'Employee': 30, 'Policy': 15, 'Person': 200, 'Account': 350}, 'relationship_counts': {'FROM_ACCOUNT': 2000, 'TO_ACCOUNT': 2000, 'COUNTERPARTY': 400, 'MADE_BY': 600, 'APPLIED_POLICY': 319, 'ABOUT': 853, 'CAUSED': 38, 'INFLUENCED': 67, 'PRECEDENT_FOR': 29, 'TRIGGERED_BY': 54, 'REGARDING': 104, 'ASSIGNED_TO': 118, 'RESOLVED_BY': 64, 'SUBMITTED_BY': 100, 'RELATED_TO': 7, 'OWNS_ACCOUNT': 52, 'OWNS': 350, 'WORKS_FOR': 140}, 'total_nodes': 3425, 'total_relationships': 7295}

=== Recent Decisions ===
{'decisions': [{'target_types': ['Organization', 'Account'], 'reasoning_summary': 'Decision made for support case. Type: rejection. Standard review completed with confidence score of ...', 'decision_type': 'rejection', 'source_system': 'Risk', 'confidence_score': 0.75, 'reasoning': 'Decision made for support case. Type: rejection. Standard review completed with con

In [34]:
print("\n=== Graph Data ===")
graph = requests.get(f'{base_url}/graph?limit=20')
graph_data = graph.json()
print(f"Nodes: {len(graph_data['nodes'])}, Relationships: {len(graph_data['relationships'])}")


=== Graph Data ===
Nodes: 20, Relationships: 298


In [35]:
from neo4j import GraphDatabase

uri = "neo4j+s://1cc5d8d5.databases.neo4j.io"
username = "neo4j"
password = "a8DQ6lCKS9R2xLjgXs22vQOhcFphBxwkPW_ZppoGOPE"

driver = GraphDatabase.driver(uri, auth=(username, password))

with driver.session(database="neo4j") as session:
    # Drop old 1536-dimension indexes
    print("Dropping old indexes...")
    session.run("DROP INDEX decision_reasoning_idx IF EXISTS")
    session.run("DROP INDEX policy_description_idx IF EXISTS")
    
    # Create new 1024-dimension indexes
    print("Creating new 1024-dimension indexes...")
    session.run("""
        CREATE VECTOR INDEX decision_reasoning_idx IF NOT EXISTS
        FOR (d:Decision) ON (d.reasoning_embedding)
        OPTIONS {indexConfig: {`vector.dimensions`: 1024, `vector.similarity_function`: 'cosine'}}
    """)
    
    session.run("""
        CREATE VECTOR INDEX policy_description_idx IF NOT EXISTS
        FOR (p:Policy) ON (p.description_embedding)
        OPTIONS {indexConfig: {`vector.dimensions`: 1024, `vector.similarity_function`: 'cosine'}}
    """)
    
    print("✅ Indexes updated to 1024 dimensions!")

driver.close()

Dropping old indexes...
Creating new 1024-dimension indexes...
✅ Indexes updated to 1024 dimensions!


In [46]:
import requests

base_url = 'http://localhost:8003/api'

# Test the credit limit increase query
response = requests.post(
    f'{base_url}/chat',
    json={
        "message": "Should we approve a credit limit increase for Jessica Norris? She's requesting a $25,000 limit increase.",
        "conversation_history": []
    }
)

print("Status Code:", response.status_code)
print("\n" + "="*60)
print("BEDROCK CLAUDE RESPONSE:")
print("="*60)

if response.status_code == 200:
    result = response.json()
    print(result['response'])
    
    # Show tool calls if any
    if 'tool_calls' in result and result['tool_calls']:
        print("\n" + "="*60)
        print("TOOLS USED:")
        print("="*60)
        for tool in result['tool_calls']:
            print(f"- {tool['name']}")
else:
    print("Error:", response.json())

Status Code: 200

BEDROCK CLAUDE RESPONSE:
I'll help you evaluate this credit limit increase request for Jessica Norris. Let me gather information about her account history and find relevant precedents.Now let me get her past decisions and look for similar credit limit increase precedents:Let me look for any fraud patterns on her accounts and check for relevant credit policies:Now let me look at the causal chain for the fraud decision to understand what led to it and what happened after:Based on my comprehensive analysis of Jessica Norris's profile and history, here is my recommendation:

## **RECOMMENDATION: CONDITIONAL APPROVAL WITH ENHANCED MONITORING**

### **Executive Summary**
I recommend **conditionally approving** a credit limit increase for Jessica Norris, but with a **reduced amount of $15,000 (instead of $25,000)** and enhanced monitoring requirements. This decision requires escalation to a manager with authorization level 4 or higher for final approval.

### **Key Factors A